In [2]:
from pathlib import Path
import shutil
import random
import os
import sys
from fastprogress import progress_bar
from scipy.ndimage.interpolation import zoom as npzoom
from fastprogress import progress_bar, master_bar
import PIL.Image
import numpy as np
import libtiff

relative_path = os.path.abspath(os.path.join('..'))
if relative_path not in sys.path:
    sys.path.append(relative_path)
from utils import get_czi_shape_info, build_index
from skimage.util import random_noise, img_as_ubyte
from skimage import filters
from pdb import set_trace

In [3]:
data_path = Path('/home/bpho/Dropbox (BPHO)')
sources = data_path/'bpho_DL_datasources'

dname = 'transfer_learning_neuron_003_oneshot'
dpath = data_path/'datasets'/dname
if dpath.exists(): shutil.rmtree(dpath)

hr_path = dpath/'hr'
lr_path = dpath/'lr'
lr_up_path = dpath/'lr_up'

#valid_pct = 0.2

In [15]:
# unpaired images (mito, fromserver, neuron, microtubule, moreNeuron)
# neuron_1 movies
neuron_1 = sources/'Live_Neuron_Data_redo_for_Gulcin_and_Fred/single image for fine tuning for low res confocal 2 series'
hr_neuron_1 = list(neuron_1.glob('*.tif'))

neuron_1_train = [fn for fn in hr_neuron_1]
len(neuron_1_train)

1

In [16]:
train_files = []
valid_files = []

In [17]:
def new_crappify(x, scale=4):
    x = random_noise(x, mode='salt', amount=0.005)
    x = random_noise(x, mode='pepper', amount=0.005)
    lvar = filters.gaussian(x, sigma=5)
    x = random_noise(x, mode='localvar', local_vars=lvar*0.5)
    x_down = npzoom(x, 1/scale, order=1)
    x_up = npzoom(x_down, scale, order=1)
    return x_down, x_up

In [28]:
def new_crappify_tif_to_tifs(tif_fn, hr_dir, lr_dir, lr_up_dir, base_name, max_scale=1.05):
    hr_dir, lr_dir, lr_up_dir = Path(hr_dir), Path(lr_dir), Path(lr_up_dir)
    hr_dir.mkdir(parents=True, exist_ok=True)
    lr_dir.mkdir(parents=True, exist_ok=True)
    lr_up_dir.mkdir(parents=True, exist_ok=True)
    
    data = libtiff.TiffFile(tif_fn)
    data = data.get_tiff_array()
    depths = data.shape[0]
    img_max = None
    preds = []
    origs = []
    for depth in progress_bar(list(range(depths))):
        save_fn = f'{base_name}_{depth:03d}.tif'
        img = data[depth].astype(np.float)
        if img_max is None: img_max = img.max() * max_scale
        img /= img_max
        down_img, down_up_img = new_crappify(img)

        img = img_as_ubyte(img)
        pimg = PIL.Image.fromarray(img, mode='L')
        cur_size = pimg.size
        pimg.save(hr_dir/save_fn)

        small_img = PIL.Image.fromarray(img_as_ubyte(down_img))
        big_img = PIL.Image.fromarray(img_as_ubyte(down_up_img))
        small_img.save(lr_dir/save_fn)
        big_img.save(lr_up_dir/save_fn)

In [29]:
for lst in [hr_neuron_1]:
    split_idx = len(lst)
    train_files += lst[:]
    for subdir, file_list in [('train', train_files),('valid', valid_files)]:
        print(f'\n\ncopy, crappify and upsample {subdir} files\n\n')
        for fn in progress_bar(file_list):
            base_name = fn.stem
            new_crappify_tif_to_tifs(
                fn, hr_path/subdir, lr_path/subdir, lr_up_path/subdir,
                base_name, max_scale=1.05)
print('done')



copy, crappify and upsample train files






copy, crappify and upsample valid files




/home/bpho/.conda/envs/fastai/lib/python3.7/site-packages/fastprogress/fastprogress.py:96: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


done
